In [2]:
import requests
from bs4 import BeautifulSoup

## Google parse

In [ ]:
url = 'https://www.google.com/search?q=amazon+a+dance+with+dragons'


ua = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'}                                                                

response = requests.get(url, headers=ua)
content = response.content

# Parse for amazon link
soup = BeautifulSoup(content, 'html.parser')

for link in soup.find_all('a'):
    print(link.get('href'))


## Amazon parse

In [2]:
    
link = 'https://www.amazon.com/Guide-Future-Medicine-Technology-Human-ebook/dp/B00N3Z7NMC'
    
ua = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'}                                                                

response = requests.get(link, headers=ua)
content = response.content

# Parse for amazon link
soup = BeautifulSoup(content, 'html.parser')

for link in soup.find_all('a'):
    print(link)

IndentationError: expected an indented block (<ipython-input-2-542ea0b9328e>, line 13)

In [10]:
#url = 'https://www.amazon.com/Guide-Future-Medicine-Technology-Human-ebook/dp/B00N3Z7NMC'
url = 'https://www.amazon.com/Dance-Dragons-Song-Ice-Fire/dp/0553582011'


ua = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'}
response = requests.get(url, headers=ua)
content = response.content

# Parse for amazon link
soup = BeautifulSoup(content, 'html.parser')






A Dance with Dragons (A Song of Ice and Fire)
George R. R. Martin
[]


In [53]:
# Title
try:
    title = soup.find_all(id='productTitle')[0].contents[0]
    #print(title)
except:
    print("couldn't find title")

    
    
    
# Author
try:
    author = soup.find_all(class_ = 'a-link-normal contributorNameID')[0].contents[0]
    #print(author)
except:
    print("couldn't find title")
    
    
# ISBN-10
try:
    book_details = soup.findAll(id = 'detail-bullets')
    
    
    for line in str(book_details[0]).split('\n'):
        if 'ISBN-10' in line:
            print(line[20:30])
        
    #print(str(book_details[0]).split('\n'))
    
    #print(str(book_details).split('\n'))
except:
    print("couldn't find isbn 13")

0553582011
